# AJUSTAR LOS DATAFRAMES Y CONECTAR CON MONGODB

# IMPORTAR LIBRERIAS 

In [1]:
import requests
import json
from dotenv import load_dotenv
import os
import pandas as pd
import pyjsonviewer
import geopandas 
import shapely
import pymongo as pym
from pymongo import MongoClient,GEOSPHERE
import shapely.geometry as shg
import geopandas as gdp

# IMPORTAR TODOS LOS DATAFRAMES 

In [2]:
london_schools =  pd.read_csv("Data/London/ln_schools.csv")
london_designing_companies =  pd.read_csv("Data/London/ln_desing.csv")
london_tech_startups =  pd.read_csv("Data/London/ln_tech_startups.csv")
london_leisure =  pd.read_csv("Data/London/ln_leisure.csv")
london_travelling_stations =  pd.read_csv("Data/London/ln_stations.csv")
london_parks =  pd.read_csv("Data/London/ln_parks.csv")
london_df_entero =  pd.read_csv("Data/London/ln_entero.csv")

In [3]:
singapore_schools =  pd.read_csv("Data/Singapore/sg_schools.csv")
singapore_designing_companies =  pd.read_csv("Data/Singapore/sg_designco.csv")
singapore_tech_startups =  pd.read_csv("Data/Singapore/sg_techstar.csv")
singapore_leisure =  pd.read_csv("Data/Singapore/sg_leisure.csv")
singapore_travelling_stations =  pd.read_csv("Data/Singapore/sg_stations.csv")
singapore_parks =  pd.read_csv("Data/Singapore/sg_parks.csv")
singapore_df_entero =  pd.read_csv("Data/Singapore/sg_entero.csv")

In [4]:
sanfrancisco_schools =  pd.read_csv("Data/SanFrancisco/sf_schools.csv")
sanfrancisco_designing_companies =  pd.read_csv("Data/SanFrancisco/sf_design.csv")
sanfrancisco_tech_startups =  pd.read_csv("Data/SanFrancisco/sf_techstartups.csv")
sanfrancisco_leisure =  pd.read_csv("Data/SanFrancisco/sf_leisure.csv")
sanfrancisco_travelling_stations =  pd.read_csv("Data/SanFrancisco/sf_travelling.csv")
sanfrancisco_parks =  pd.read_csv("Data/SanFrancisco/sf_parks.csv")
sanfrancisco_df_entero =  pd.read_csv("Data/SanFrancisco/sf_entero.csv")

# CAMBIAR DATAFRAMES Y METERLOS EN MONGODB

In [5]:
london_df_entero.sample(3)

,Unnamed: 0,Name,Classification,Type,Country,Adress,Latitude,Longitude,location
145,145,XOYO,Leisure Places,Nightclub,GB,32-37 Cowper St,51.525472,-0.085731,"{'type': 'Point', 'coordinates': (51.52547227,..."
177,177,The Princess of Shoreditch,Leisure Places,Pub,GB,76 Paul St,51.525393,-0.083895,"{'type': 'Point', 'coordinates': (51.525393024..."
350,350,Tower Hill,Parks,Neighborhood,GB,London,51.510053,-0.076536,"{'type': 'Point', 'coordinates': (51.510052558..."


In [6]:
singapore_df_entero.sample(3)

,Unnamed: 0,Name,Classification,Type,Country,Adress,Latitude,Longitude,location
193,193,The SMU Shop,Leisure Places,Gift Shop,SG,Singapore,1.296425,103.849889,"{'type': 'Point', 'coordinates': (1.2964253900..."
298,298,Sticky fingers,Leisure Places,Irish Pub,SG,The riverside plaza (11 Keng Cheow St),1.288875,103.843887,"{'type': 'Point', 'coordinates': (1.2888747302..."
26,26,PSB Academy Block A,School,High School,SG,169199,1.287704,103.825857,"{'type': 'Point', 'coordinates': (1.2877039453..."


In [7]:
sanfrancisco_df_entero.sample(3)

,Unnamed: 0,Name,Classification,Type,Country,Adress,Latitude,Longitude,location
139,139,pandadoc,Technological Startups,Tech Startup,US,565 Commercial St (Montgomery St),37.79391,-122.402772,"{'type': 'Point', 'coordinates': (37.793910390..."
302,302,Cable Car Stop Pine And Powell,Travelling Stations,Light Rail Station,US,"San Francisco, CA 94108",37.79121,-122.408940,"{'type': 'Point', 'coordinates': (37.79121, -1..."
135,135,Silkroad Technology - San Francisco,Technological Startups,Tech Startup,US,550 Montgomery St,37.79421,-122.402927,"{'type': 'Point', 'coordinates': (37.794209892..."


In [8]:
print(type(sanfrancisco_df_entero['location']))

<class 'pandas.core.series.Series'>


# LONDON

In [9]:
ln1 = london_df_entero.drop(['Unnamed: 0'], axis=1)

In [10]:
sf1 = sanfrancisco_df_entero.drop(['Unnamed: 0'], axis=1)

In [11]:
sg1 = singapore_df_entero.drop(['Unnamed: 0'], axis=1)

In [12]:
#LONDON

In [13]:
ln1.head(1)

,Name,Classification,Type,Country,Adress,Latitude,Longitude,location
0,Hatching Dragons,School,Nursery School,GB,1 Viscount Street,51.521599,-0.095356,"{'type': 'Point', 'coordinates': (51.521599411..."


In [14]:
gdf = gdp.GeoDataFrame(ln1, geometry= gdp.points_from_xy(ln1.Latitude, ln1.Longitude ))
gdf.columns=['Name', 'Classification','Type','Country','Adress', "Latitude", "Longitude", "location", "location2" ]
#Lo aplicamos a toda la columna
gdf['location2']= gdf['location2'].apply(lambda x:shapely.geometry.mapping(x))

In [15]:
client = MongoClient()
db = client.Ciudades
collection = db.create_collection(name = "London")
collection = db["London"]
collection.create_index([("location2", "2dsphere")])
data = gdf.to_dict(orient='records')
collection.insert_many(data)

In [16]:
# SINGAPORE

In [17]:
gdf = gdp.GeoDataFrame(sg1, geometry= gdp.points_from_xy(sg1.Longitude, sg1.Latitude ))
gdf.columns=['Name', 'Classification','Type','Country','Adress', "Latitude", "Longitude", "location", "location2" ]
#Lo aplicamos a toda la columna
gdf['location2']= gdf['location2'].apply(lambda x:shapely.geometry.mapping(x))

In [18]:
collection = db.create_collection(name = "Singapore")
collection = db["Singapore"]
collection.create_index([("location2", "2dsphere")])
data1 = gdf.to_dict(orient='records')
collection.insert_many(data1)

In [19]:
# SAN FRANCISCO

In [20]:
gdf = gdp.GeoDataFrame(sf1, geometry= gdp.points_from_xy(sf1.Longitude, sf1.Latitude ))
gdf.columns=['Name', 'Classification','Type','Country','Adress', "Latitude", "Longitude", "location", "location2" ]
#Lo aplicamos a toda la columna
gdf['location2']= gdf['location2'].apply(lambda x:shapely.geometry.mapping(x))

In [21]:
collection = db.create_collection(name = "San Francisco")
collection = db["San Francisco"]
collection.create_index([("location2", "2dsphere")])
data2 = gdf.to_dict(orient='records')
collection.insert_many(data2)